In [1]:
import pandas as pd
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
data = pd.read_json('sherdog/data/cumulative.json')

In [11]:
def parse_date(timeinmilis):
    try:
        timestamp = timeinmilis / 1000
        return datetime.utcfromtimestamp(timestamp)
    except:
        return None

    
def flatten(data):
    X = pd.DataFrame()
    # X['date'] = data['date']
    X['target'] = data['result'].map({'win': 1, 'loss': 0})
    for fighter in ['fighter', 'opponent']:
        for key in ['birth', 'started']:
            curr = '{}_{}'.format(fighter, key)
            # X[curr] = data[fighter].apply(lambda x: parse_date(x[key]))

        # Cumulative stats
        for key in ['loss', 'win']:
            stats = 'cumulative'
            for result in ['loss', 'win']:
                for method in ['decision', 'knockout', 'submission', 'total']:
                    curr = '{}_{}_{}_{}_{}'.format(fighter, stats, key, result, method)
                    X[curr] = data[fighter].apply(lambda x: x[stats][key][result][method])
                    X[curr] = X[curr].apply(lambda x: float(x))

        # Historical stats
        stats = 'history'
        for result in ['loss', 'win']:
            for method in ['decision', 'knockout', 'submission', 'total']:
                curr = '{}_{}_{}_{}'.format(fighter, stats, result, method)
                X[curr] = data[fighter].apply(lambda x: x[stats][result][method])
                X[curr] = X[curr].apply(lambda x: float(x))

        for key in ['fights', 'positions', 'time']:
            curr = '{}_{}_{}'.format(fighter, stats, key)
            X[curr] = data[fighter].apply(lambda x: x[stats][key])
            X[curr] = X[curr].apply(lambda x: float(x))
    
    return X


def get_data(data):
    x = flatten(data)
    y = x['target']
    x.drop(['target'], axis=1, inplace=True)
    x.fillna(0, inplace=True)
    return train_test_split(x, y)

In [12]:
X_train, X_test, y_train, y_test = get_data(data)

In [13]:
X_train.head()

,fighter_cumulative_loss_loss_decision,fighter_cumulative_loss_loss_knockout,fighter_cumulative_loss_loss_submission,fighter_cumulative_loss_loss_total,fighter_cumulative_loss_win_decision,fighter_cumulative_loss_win_knockout,fighter_cumulative_loss_win_submission,fighter_cumulative_loss_win_total,fighter_cumulative_win_loss_decision,fighter_cumulative_win_loss_knockout,...,opponent_history_loss_knockout,opponent_history_loss_submission,opponent_history_loss_total,opponent_history_win_decision,opponent_history_win_knockout,opponent_history_win_submission,opponent_history_win_total,opponent_history_fights,opponent_history_positions,opponent_history_time
104399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,2.0,0.0,2.0,1.0,2.0,1.0,4.0,6.0,39.0,18.233333
4972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
335833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,8.0,0.000000
70746,1.0,2.0,5.0,8.0,0.0,0.0,8.0,8.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,6.0,0.000000
231311,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,2.0,2.0,3.0,12.0,12.050000


In [24]:
input_dim = X_train.shape[1]
model = Sequential()
model.add(Dense(256, input_dim=input_dim, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [25]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
model.fit(X_train.values, y_train, epochs=150, batch_size=1024)

Epoch 1/150
321460/321460 [==============================] - 2s - loss: 0.6280 - acc: 0.7217      

In [27]:
scores = model.evaluate(X_test.values, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

107040/107154 [============================>.] - ETA: 0s  